In [23]:
import torch
import torch.nn as nn
import torch.optim as optim

from model_factories import get_golem_model, get_resnet18_model, get_resnet34_model, get_vitbase_model
from dataloaders import get_dataloaders
from model_utils import train_epoch, eval

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader, test_loader, classes = get_dataloaders()
gc = get_resnet18_model()
# gc = get_golem_model() --> needs fixing
gc.to(device)

GolemClassifier(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tra

# Scenario 1

In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(gc.parameters(), lr=1e-3)

In [26]:
for i in range(2):
    loss = train_epoch(gc, criterion, optimizer, train_loader, device)
    print(f"{i+1}/2: loss={loss}")

1/2: loss=2324.447230517864
2/2: loss=1660.8864401876926


In [27]:
loss, accuracy, precision, recall, f1 = eval(gc, criterion, test_loader, device)
print(f"Avg loss {loss}\nAccu {accuracy}")

RuntimeError: The size of tensor a (10) must match the size of tensor b (32) at non-singleton dimension 1

# Scenario 2

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(gc.parameters(), lr=1e-3, momentum=0.9)

In [18]:
for i in range(2):
    loss = train_epoch(gc, criterion, optimizer, train_loader, device)
    print(f"{i+1}/2: loss={loss}")

RuntimeError: Given groups=1, weight of size [32, 1, 3, 3], expected input[32, 3, 32, 32] to have 1 channels, but got 3 channels instead

In [31]:
loss, accuracy, precision, recall, f1 = eval(gc, criterion, test_loader, device)
print(f"Avg loss {loss}\nAccu {accuracy}")

Avg loss 6721
Accu 10000


# Scenario 3

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(gc.parameters(), lr=1e-3, momentum=0.9)

In [ ]:
for i in range(2):
    loss = train_epoch(gc, criterion, optimizer, train_loader, device)
    print(f"{i+1}/2: loss={loss}")

In [ ]:
loss, accuracy, precision, recall, f1 = eval(gc, criterion, test_loader, device)
print(f"Avg loss {loss}\nAccu {accuracy}")

Avg loss 6721
Accu 10000
